In [36]:
import torch
from trainer import *
from torch.utils.data import DataLoader
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from models.CNN import StrainEnergyCANN_Ani
from sklearn.metrics import r2_score
from typing import Optional

# Предобработка данных

In [37]:
path_to_data = r"..\..\data\GoreTex"
# tensl_data_path = r"..\..\data\PDMS\Shear_pdms.csv"
# shear_data_path = r"..\..\data\PDMS\Tensile_pdms.csv"
# experiments_path = [compr_data_path, tensl_data_path, shear_data_path]

In [56]:
def get_list_of_paths_to_experiments_type(experiment="uniaxial"):
    experiment_type_path = os.path.join(path_to_data, experiment)
    l = []
    for file in os.listdir(experiment_type_path):
        l.append(os.path.join(experiment_type_path, file))
    return l

def load_and_extract(file_path, experiment_type):
    df = pd.read_csv(file_path)
    df['experiment_type'] = experiment_type
    return df[['lambda_clamps_X', 'lambda_clamps_Y', 'mean_stress_x_mpa', 'mean_stress_y_mpa', 'experiment_type']]

get_list_of_paths_to_experiments_type()

['..\\..\\data\\GoreTex\\uniaxial\\GoreTex09mmUniX.csv',
 '..\\..\\data\\GoreTex\\uniaxial\\GoreTex09mmUniY.csv']

In [59]:
experiment = "uniaxial"
experiments_path = get_list_of_paths_to_experiments_type(experiment)
data_frames = [load_and_extract(file, file[-11:-4]) for file in experiments_path]

# print(data_frames)

df = pd.concat(data_frames, ignore_index=True)
thinned_data_frames = []
num_points = 100

sampled_df_list = []

for df in data_frames:
    indices = np.linspace(10, len(df) - 1, num_points, dtype=int)
    # df[1] = df[1] / 10**6
    sampled_df = pd.DataFrame(df.iloc[indices].copy())
    # print(type(sampled_df))
    sampled_df['lambdas'] = list(zip(sampled_df['lambda_clamps_X'], sampled_df['lambda_clamps_Y']))
    sampled_df['stresses'] = list(zip(sampled_df['mean_stress_x_mpa'], sampled_df['mean_stress_y_mpa']))
    sampled_df_list.append(sampled_df[:50])
    # thinned_df = df.iloc[::len(df) // 20, :]  # Выбор каждого 45-го значения
    # thinned_data_frames.append(thinned_df)
data_frames = sampled_df_list
data_frames = data_frames
# sampled_df_list[0]


In [60]:
I1_bx = lambda lam1, lam2: lam1**2 + lam2**2 + 1 / (lam1 * lam2)**2
I2_bx = lambda lam1, lam2: 1 / lam1**2 + 1 / lam2**2 + (lam1 * lam2)**2
I4_bx = lambda lam1, lam2: lam1**2  + math.cos(math.pi / 4)**2 + lam2 * math.sin(torch.pi / 4)**2
I5_bx = lambda lam1, lam2: lam1**2  + math.cos(math.pi / 4)**4 + lam2 * math.sin(torch.pi / 4)**4

F_bx = lambda lam1, lam2: ([lam1, 0, 0], [0, lam2, 0], [0, 0, 1 / (lam1 * lam2)])


In [61]:
mechanical_variables = {
    "I1": I1_bx,
    "I2": I2_bx,
    "I4": I4_bx,
    "I5": I5_bx,

    "F": F_bx,
    # "exp_type": [(lambda x: 1), (lambda x: 0)] # 1 - torsion&compression, 0 - shear
    # "torsion_compression": (lambda x: 1)
}

# calculate I1, I2, F from lambda (torsion&compression and shear)
for variable in mechanical_variables.keys():
    func_calc = mechanical_variables.get(variable)

    for data_frame in data_frames:
        data_frame[variable] = data_frame['lambdas'].apply(lambda lambdas: func_calc(lambdas[0], lambdas[0]))


C:\Users\User\AppData\Local\Temp\ipykernel_10820\2117353720.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[variable] = data_frame['lambdas'].apply(lambda lambdas: func_calc(lambdas[0], lambdas[0]))
C:\Users\User\AppData\Local\Temp\ipykernel_10820\2117353720.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[variable] = data_frame['lambdas'].apply(lambda lambdas: func_calc(lambdas[0], lambdas[0]))
C:\Users\User\AppData\Local\Temp\ipykernel_10820\2117353720.py:17: SettingWithCopy

In [63]:
experiments = ['Compression', 'Tensile', 'Shear']
combined_data = pd.concat(data_frames).reset_index(drop=True, inplace=False)
# combined_data.columns = ['lambda1', 'P_experimental', 'I1', 'I2', 'F', 'experiment_type']
combined_data.pop("lambdas")
combined_data.pop("stresses")
experiment_type = combined_data.pop("experiment_type")
combined_data["experiment_type"] = experiment_type
combined_data
combined_data.to_csv( "Uniaxial.csv")
# combined_data = combined_data["lambda_clamps_X", "lambda_clamps_Y",	"mean_stress_x_mpa", "mean_stress_y_mpa", "experiment_type" 

In [43]:
# new_combined_data = pd.DataFrame(columns=["lambdas, 

In [44]:
from torch.utils.data import DataLoader, random_split, Dataset, TensorDataset
import copy 

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        # self.features = [dataframe[0],dataframe[2], dataframe[3], dataframe[4], dataframe[5]]
        # self.targets  = dataframe[1]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = copy.deepcopy([*self.data.iloc[idx]])
        target1 = features.pop(2)
        target2 = features.pop(2)
        target = torch.tensor([target1, target2])
        return features, target

    def to_tensor(self):
        for column in self.data.columns:
            if column != "experiment_type":
                self.data[column] = self.data[column].apply(
                    lambda x: torch.tensor(x, dtype=torch.float32)).copy()


In [45]:
start = 0
end = -1
train_dataset = CustomDataset(combined_data[start:end].copy())
test_dataset = CustomDataset(combined_data.copy())
f, t = train_dataset[1]
# lam, i1, i2, F, exp_type = f
print(f)
print(t)

[1.0571428571428572, 1.0571428571428572, 3.0357936569581625, 3.038547748518468, 2.1461224489795923, 1.6318367346938776, ([1.0571428571428572, 0, 0], [0, 1.0571428571428572, 0], [0, 0, 0.8948137326515704]), '100_100']
tensor([0.0023, 0.0262], dtype=torch.float64)


In [46]:
def init_loaders(experiments:Optional[str]=["Shear", "Tensile", "Comression"]):
    if type(experiments) == str:
        experiments = [experiments]
    elif type(experiments) == list:
        df = pd.concat([combined_data[combined_data["experiment_type"] == experiment] for experiment in experiments]).reset_index(drop=True, inplace=False)
    else:
        df=combined_data
    train_dataset = CustomDataset(df.copy())
    test_dataset = CustomDataset(combined_data.copy())
    
    train_dataset.to_tensor()
    test_dataset.to_tensor()
    
    train_data_loader = DataLoader(
                            train_dataset,
                            shuffle=True,
                            # num_workers=1,
                            pin_memory=False
    )
    test_data_loader = DataLoader(
                            test_dataset,
                            shuffle=False,
                            # num_workers=1,
                            pin_memory=False
    )

    return train_data_loader, test_data_loader 

In [47]:
train_data_loader, test_data_loader = init_loaders(None)
train_data_loader.dataset.data

,lambda_clamps_X,lambda_clamps_Y,mean_stress_x_mpa,mean_stress_y_mpa,I1,I2,I4,I5,F,experiment_type
0,tensor(1.0571),tensor(1.0571),tensor(0.0023),tensor(0.0262),tensor(3.0358),tensor(3.0385),tensor(2.1461),tensor(1.6318),"[[tensor(1.0571), tensor(0.), tensor(0.)], [te...",100_100
1,tensor(1.0571),tensor(1.0571),tensor(0.0023),tensor(0.0262),tensor(3.0358),tensor(3.0385),tensor(2.1461),tensor(1.6318),"[[tensor(1.0571), tensor(0.), tensor(0.)], [te...",100_100
2,tensor(1.0629),tensor(1.0629),tensor(0.0027),tensor(0.0330),tensor(3.0429),tensor(3.0466),tensor(2.1611),tensor(1.6454),"[[tensor(1.0629), tensor(0.), tensor(0.)], [te...",100_100
3,tensor(1.0686),tensor(1.0686),tensor(0.0032),tensor(0.0397),tensor(3.0507),tensor(3.0554),tensor(2.1761),tensor(1.6590),"[[tensor(1.0686), tensor(0.), tensor(0.)], [te...",100_100
4,tensor(1.0743),tensor(1.0743),tensor(0.0038),tensor(0.0480),tensor(3.0590),tensor(3.0649),tensor(2.1912),tensor(1.6727),"[[tensor(1.0743), tensor(0.), tensor(0.)], [te...",100_100
...,...,...,...,...,...,...,...,...,...,...
345,tensor(1.2857),tensor(1.0714),tensor(0.3181),tensor(0.8163),tensor(3.6721),tensor(3.9425),tensor(2.7959),tensor(2.2245),"[[tensor(1.2857), tensor(0.), tensor(0.)], [te...",100_033
346,tensor(1.2800),tensor(1.0700),tensor(0.2889),tensor(0.7761),tensor(3.6493),tensor(3.9051),tensor(2.7784),tensor(2.2084),"[[tensor(1.2800), tensor(0.), tensor(0.)], [te...",100_033
347,tensor(1.2743),tensor(1.0686),tensor(0.2541),tensor(0.7219),tensor(3.6269),tensor(3.8684),tensor(2.7609),tensor(2.1924),"[[tensor(1.2743), tensor(0.), tensor(0.)], [te...",100_033
348,tensor(1.2686),tensor(1.0671),tensor(0.2276),tensor(0.6891),tensor(3.6047),tensor(3.8326),tensor(2.7436),tensor(2.1764),"[[tensor(1.2686), tensor(0.), tensor(0.)], [te...",100_033


# Обучение модели

In [48]:
def plot_results_by_experiment_type(data: pd.DataFrame, plot_name_prefix="plot"):
    
    # Переименовываем столбцы для удобства
    data.columns = ['index', 'lambda_x', 'lambda_y', 'stress_x', 'stress_y', 'I1', 'I2', 'I4', 'I5', 'F', 'experiment_type', 'P11', 'P22']

    # Получим уникальные типы экспериментов
    experiment_types = data['experiment_type'].unique()
    
    for experiment in experiment_types:
        subset = data[data['experiment_type'] == experiment]
        
        # Создадим первый график (lambda_x, P11) и скаттер на нем stress_x для данного типа эксперимента
        fig, ax1 = plt.subplots(figsize=(10, 6))
        
        sns.lineplot(data=subset, x='lambda_x', y='P11', ax=ax1, label='P11')
        sns.scatterplot(data=subset, x='lambda_x', y='stress_x', ax=ax1, color='red', label='Stress_x')
        
        # Рассчитаем R² для P11
        r2_p11 = r2_score(subset['stress_x'], subset['P11'])
        
        ax1.set_title(f'{experiment}: P11 and Stress_x\nR² = {r2_p11:.2f}')
        ax1.set_xlabel('Lambda_x')
        ax1.set_ylabel('P11 / Stress_x (MPa)')
        plt.legend()
        plt.savefig(f"{plot_name_prefix}_{experiment}_plot1.png")
        plt.show()
        
        # Создадим второй график (lambda_y, P22) и скаттер на нем stress_y для данного типа эксперимента
        fig, ax2 = plt.subplots(figsize=(10, 6))
        
        sns.lineplot(data=subset, x='lambda_y', y='P22', ax=ax2, label='P22')
        sns.scatterplot(data=subset, x='lambda_y', y='stress_y', ax=ax2, color='blue', label='Stress_y')
        
        # Рассчитаем R² для P22
        r2_p22 = r2_score(subset['stress_y'], subset['P22'])
        
        ax2.set_title(f'{experiment}: P22 and Stress_y\nR² = {r2_p22:.2f}')
        ax2.set_xlabel('Lambda_y')
        ax2.set_ylabel('P22 / Stress_y (MPa)')
        plt.legend()
        plt.savefig(f"{plot_name_prefix}_{experiment}_plot2.png")
        plt.show()

# Вызовем функцию plot_results_by_experiment_type с нашими данными


In [49]:
from collections import OrderedDict
import torch

# Provided weights
weights = [[-0.00000000e+00, -0.00000000e+00],
 [ 8.06281149e-01,  1.40696345e-03],
 [-0.00000000e+00, -0.00000000e+00],
 [ 6.46267161e-02,  2.84846843e-04],
 [-0.00000000e+00, -0.00000000e+00],
 [ 4.88660574e-01,  1.02614926e-03],
 [-0.00000000e+00, -0.00000000e+00],
 [ 1.09509790e+00,  1.44241273e-03],
 [ 1.32167622e-01,  2.50902846e-02],
 [ 6.15145266e-01,  1.99771114e-03],
 [ 1.48086607e+00,  8.69683549e-02],
 [ 4.96627361e-01,  1.48953160e-03],
 [ 1.23559475e+00,  9.31678526e-03],
 [ 9.03831899e-01,  1.09326339e-03],
 [-0.00000000e+00, -0.00000000e+00],
 [ 1.22360349e+00,  6.04832545e-04]]

weights = np.array(weights).transpose().reshape(32)
# Convert weights to tensors
weight_tensors = [torch.tensor([[w]]) for w in weights]

# Create OrderedDict
model_weights = OrderedDict([
    ('single_inv_net1.w11.weight', weight_tensors[0]),
    ('single_inv_net1.w21.weight', weight_tensors[1]),
    ('single_inv_net1.w31.weight', weight_tensors[2]),
    ('single_inv_net1.w41.weight', weight_tensors[3]),
    ('single_inv_net2.w11.weight', weight_tensors[4]),
    ('single_inv_net2.w21.weight', weight_tensors[5]),
    ('single_inv_net2.w31.weight', weight_tensors[6]),
    ('single_inv_net2.w41.weight', weight_tensors[7]),
    ('single_inv_net4.w11.weight', weight_tensors[8]),
    ('single_inv_net4.w21.weight', weight_tensors[9]),
    ('single_inv_net4.w31.weight', weight_tensors[10]),
    ('single_inv_net4.w41.weight', weight_tensors[11]),
    ('single_inv_net5.w11.weight', weight_tensors[12]),
    ('single_inv_net5.w21.weight', weight_tensors[13]),
    ('single_inv_net5.w31.weight', weight_tensors[14]),
    ('single_inv_net5.w41.weight', weight_tensors[15]),
    ('wx2.weight', torch.tensor([weight_tensors[16:]]))
])

# Display the model weights
for key, value in model_weights.items():
    print(f"{key}: {value}")


single_inv_net1.w11.weight: tensor([[-0.]], dtype=torch.float64)
single_inv_net1.w21.weight: tensor([[0.8063]], dtype=torch.float64)
single_inv_net1.w31.weight: tensor([[-0.]], dtype=torch.float64)
single_inv_net1.w41.weight: tensor([[0.0646]], dtype=torch.float64)
single_inv_net2.w11.weight: tensor([[-0.]], dtype=torch.float64)
single_inv_net2.w21.weight: tensor([[0.4887]], dtype=torch.float64)
single_inv_net2.w31.weight: tensor([[-0.]], dtype=torch.float64)
single_inv_net2.w41.weight: tensor([[1.0951]], dtype=torch.float64)
single_inv_net4.w11.weight: tensor([[0.1322]], dtype=torch.float64)
single_inv_net4.w21.weight: tensor([[0.6151]], dtype=torch.float64)
single_inv_net4.w31.weight: tensor([[1.4809]], dtype=torch.float64)
single_inv_net4.w41.weight: tensor([[0.4966]], dtype=torch.float64)
single_inv_net5.w11.weight: tensor([[1.2356]], dtype=torch.float64)
single_inv_net5.w21.weight: tensor([[0.9038]], dtype=torch.float64)
single_inv_net5.w31.weight: tensor([[-0.]], dtype=torch.floa

In [54]:
trained_model =StrainEnergyCANN_Ani()
# trained_model.load_state_dict(torch.load(r"C:\Users\User\PycharmProjects\data-driven-constitutive-modelling\src\CANN_torch\pretrained_models\GoreTex_StrainEnergyCANN_Ani\20240530_0108_206.pth"))
trained_model.load_state_dict(model_weights)
trained_model.state_dict()
trained_model.get_potential()

tensor([[-0.0000e+00, 8.0628e-01, -0.0000e+00, 6.4627e-02, -0.0000e+00, 4.8866e-01,
         -0.0000e+00, 1.0951e+00, 1.3217e-01, 6.1515e-01, 1.4809e+00, 4.9663e-01,
         1.2356e+00, 9.0383e-01, -0.0000e+00, 1.2236e+00],
        [-0.0000e+00, 1.4070e-03, -0.0000e+00, 2.8485e-04, -0.0000e+00, 1.0261e-03,
         -0.0000e+00, 1.4424e-03, 2.5090e-02, 1.9977e-03, 8.6968e-02, 1.4895e-03,
         9.3168e-03, 1.0933e-03, -0.0000e+00, 6.0483e-04]])


'0.000000 * (I1 - 3) \\\\               + 0.001407 * (e^{  0.806281 * (I1 - 3)} - 1)\\\\               + 0.000000 * (I1 - 3) ^ 2 \\\\               + 0.000285 * (e^{  0.064627 * (I1 - 3) ^ 2} - 1)\\\\                               + 0.000000 * (I2 - 3) \\\\               + 0.001026 * (e^{  0.488661 * (I2 - 3))} - 1)\\\\               + 0.000000 * (I2 - 3) ^ 2 \\\\               + 0.001442 * (e^{  1.095098 * (I2 - 3) ^ 2)} - 1) \\\\                               + 0.003316 * (I4 - 3) \\\\                + 0.001998 * (e^{  0.615145 * (I4 - 3)} - 1)\\\\                + 0.128788 * (I4 - 3) ^ 2 \\\\                + 0.001490 * (e^{  0.496627 * (I4 - 3) ^ 2} - 1)\\\\                                + 0.011512 * (I5 - 3) \\\\                + 0.001093 * (e^{  0.903832 * (I5 - 3))} - 1)\\\\                + 0.000000 * (I5 - 3) ^ 2 \\\\                + 0.000605 * (e^{  1.223603 * (I5 - 3) ^ 2)} - 1)\\\\ '

$ 0.001407 * (e^{  0.806281 * (I1 - 3)} - 1)\\\\ + 0.000285 * (e^{  0.064627 * (I1 - 3) ^ 2} - 1)\\\\ + 0.001026 * (e^{  0.488661 * (I2 - 3))} - 1)\\\\ + 0.001442 * (e^{  1.095098 * (I2 - 3) ^ 2)} - 1) \\\\                               + 0.003316 * (I4 - 3) \\\\                + 0.001998 * (e^{  0.615145 * (I4 - 3)} - 1)\\\\                + 0.128788 * (I4 - 3) ^ 2 \\\\                + 0.001490 * (e^{  0.496627 * (I4 - 3) ^ 2} - 1)\\\\                                + 0.011512 * (I5 - 3) \\\\                + 0.001093 * (e^{  0.903832 * (I5 - 3))} - 1)\\\\ + 0.000605 * (e^{  1.223603 * (I5 - 3) ^ 2)} - 1)\\\\$

In [65]:
# trained_model.load_state_dict(torch.load(r"C:\Users\User\PycharmProjects\data-driven-constitutive-modelling\src\CANN_torch\pretrained_models\GoreTex_StrainEnergyCANN_Ani\20240530_0108_206.pth"))
vpredictions = []
vtargets = []
for data in test_data_loader:
    features, target = data
    # vpredictions.append(zip(trained_model(features).detach().numpy()))
    # print(trained_model(features))
    vpredictions.append(trained_model(features).detach().numpy())
# print(trained_model.get_potential())
print(trained_model.potential_constants)
vpredictions = np.array(vpredictions)
print(vpredictions.transpose()[0] - vpredictions.transpose()[1])
combined_data["P11_model_" + name] = vpredictions.transpose()[0]
combined_data["P22_model_" + name] = vpredictions.transpose()[1]
# combined_data.to_csv(os.path.join(os.path.join(path, str(name)), "data.csv"))
# plot_results_by_experiment_type("data.csv")
combined_data.pop("P11_model_" + name)
combined_data.pop("P22_model_" + name)

tensor([[-0.0000e+00, 8.0628e-01, -0.0000e+00, 6.4627e-02, -0.0000e+00, 4.8866e-01,
         -0.0000e+00, 1.0951e+00, 1.3217e-01, 6.1515e-01, 1.4809e+00, 4.9663e-01,
         1.2356e+00, 9.0383e-01, -0.0000e+00, 1.2236e+00],
        [-0.0000e+00, 1.4070e-03, -0.0000e+00, 2.8485e-04, -0.0000e+00, 1.0261e-03,
         -0.0000e+00, 1.4424e-03, 2.5090e-02, 1.9977e-03, 8.6968e-02, 1.4895e-03,
         9.3168e-03, 1.0933e-03, -0.0000e+00, 6.0483e-04]])
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e

ValueError: Length of values (350) does not match length of index (100)

In [64]:
data = pd.read_csv(r"C:\Users\User\PycharmProjects\data-driven-constitutive-modelling\src\CANN_torch\uniaxial.csv")
plot_results_by_experiment_type(data)

ValueError: Length mismatch: Expected axis has 11 elements, new values have 13 elements

In [53]:
StrainEnergyCANN_Ani.__name__ = "test"
StrainEnergyCANN_Ani.__name__

'test'

$psi =   0.000000 * (I1 - 3) \\               + 0.026245 * (e^{  0.000000 * (I1 - 3)} - 1)\\               - 0.025429 * ln(1 - 0.052720 * (I1 - 3)) \\               + 0.000000 * (I1 - 3) ^ 2 \\               + 0.085033 * (e^{  0.000000 * (I1 - 3) ^ 2} - 1)\\               - 0.077004 * ln(1 - 0.046803 * (I1 - 3) ^ 2) \\                               + 0.006244 * (I2 - 3) \\               + 0.046553 * (e^{  0.060594 * (I2 - 3))} - 1)\\               - 0.123550* ln(1 - 0.126146 *  (I2 - 3)) \\               + 0.000000 * (I2 - 3) ^ 2 \\               + 0.132380 * (e^{  0.051048 *(I2 - 3) ^ 2)} - 1)\\               - 0.018332* ln(1 - 0.074555 * (I2 - 3) ^ 2)\\$